#### Metrics for three interpretability techniques: LIME, ANCHOR, CIU

Metrics tested are identity, separability, fidelity, and speed.

In [2]:
import os
import time
import tqdm
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from pymfe.mfe import MFE
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

from anchor import utils
from anchor import anchor_tabular

import metrics_rules

In [24]:
# Define the path to the datasets folder
datasets_folder = "../datasets"

# Initialize empty lists to store dataframes for each file
folder_names = []
attribute_names_list = []
categorical_indicator_list = []
X_list = []
y_list = []

# Loop through each folder in the datasets folder
for folder_name in os.listdir(datasets_folder):
    folder_path = os.path.join(datasets_folder, folder_name)
    
    # Check if it's a directory
    if os.path.isdir(folder_path):
        # Construct file paths for each CSV file in the folder
        attribute_names_path = os.path.join(folder_path, "attribute_names.csv")
        categorical_indicator_path = os.path.join(folder_path, "categorical_indicator.csv")
        X_path = os.path.join(folder_path, "X.csv")
        y_path = os.path.join(folder_path, "y.csv")
        
        # Read each CSV file into a pandas dataframe
        attribute_names_df = pd.read_csv(attribute_names_path)
        categorical_indicator_df = pd.read_csv(categorical_indicator_path)
        X_df = pd.read_csv(X_path)
        y_df = pd.read_csv(y_path)
        
        # Append dataframes to the lists
        attribute_names_list.append(attribute_names_df)
        categorical_indicator_list.append(categorical_indicator_df)
        X_list.append(X_df)
        y_list.append(y_df)

        # Save folder name to list
        folder_names.append(folder_name)

# Subsetting for less expensive runs
X_list = [df.head(50) for df in X_list]
y_list = [df.head(50) for df in y_list]

# For testing the techniques
X = X_list[:40]
y = y_list[:40]

# Names of chosen datasets
X_folder_names = folder_names[:40]

# For testing later
X_list_test = X_list[-10:]
y_list_test = y_list[-10:]

# Names of the test datasets
X_folder_names_test = folder_names[-10:]

In [25]:
# Preprocessing
def convert_to_numeric_and_impute(X_list, y_list):
    imputer = SimpleImputer(strategy='mean')
    label_encoder = LabelEncoder()

    def process_dataframe(df):
        for column in df.columns:
            if isinstance(df[column].iloc[0], csr_matrix):
                df[column] = df[column].apply(lambda x: x.toarray()[0,0] if x.shape[1] == 1 else x.toarray())

            df[column] = pd.to_numeric(df[column], errors='coerce')

            if df[column].dtype == 'object':
                # Fill NaN with a placeholder and then label encode
                df[column] = df[column].fillna('Missing')
                df[column] = label_encoder.fit_transform(df[column])
            else:
                if df[column].notna().any():
                    df[column] = imputer.fit_transform(df[[column]]).ravel()
                else:
                    df[column] = df[column].fillna(0)

        return df

    X_list = [process_dataframe(df) for df in X_list]
    y_list = [process_dataframe(df) for df in y_list]

    return X_list, y_list

X, y = convert_to_numeric_and_impute(X, y)
X_list_test, y_list_test = convert_to_numeric_and_impute(X_list_test, y_list_test)

In [6]:
# Metadata generation

# Check all available meta-features in the package
# print(MFE.valid_metafeatures()) # <- should choose more?????

columns = ['attr_to_inst',  'cat_to_num',  'freq_class.mean',  'freq_class.sd',  'inst_to_attr',  'max.mean',  'max.sd',  'min.mean',  'min.sd',  'nr_cor_attr',  'nr_norm',  'sd.mean',  'sd.sd']

metadata_df = pd.DataFrame(columns=columns)

for i in range(len(X)):

    mfe = MFE(features=["attr_to_inst", "cat_to_num", "freq_class", "inst_to_attr", "sd", "nr_norm", "nr_cor_attr", "min", "max"])
    mfe.fit(np.array(X[i]), np.array(y[i]))
    ft = mfe.extract(
        sd={"ddof": 0},
        nr_norm={"method": "all", "failure": "hard", "threshold": 0.025},
        nr_cor_attr={"threshold": 0.6},
    )

    new = pd.DataFrame(np.array(ft[1]).reshape(1, -1), columns=ft[0])
    metadata_df = metadata_df.append(new, ignore_index=True)

metadata_df['folder'] = X_folder_names
metadata_df['folder'] = metadata_df['folder'].astype(int)
metadata_df.head()

,attr_to_inst,cat_to_num,freq_class.mean,freq_class.sd,inst_to_attr,max.mean,max.sd,min.mean,min.sd,nr_cor_attr,nr_norm,sd.mean,sd.sd,folder
0,0.10,0.0,0.5,0.08,10.000000,728712.000000,894485.604730,1241.000000,2475.504151,0.100000,0.0,233606.804592,286748.820769,1046
1,0.74,0.0,0.5,0.44,1.351351,10401.002973,58049.483653,0.223514,0.470010,0.223724,1.0,1533.639358,8540.628907,1049
2,0.74,0.0,0.5,0.44,1.351351,9470.734324,51785.848076,0.437568,0.715264,0.340841,0.0,1347.550690,7361.261968,1050
3,0.42,0.0,0.5,0.48,2.380952,47852.292857,199422.479258,0.238095,0.425918,0.419048,0.0,6983.118814,28979.323785,1053
4,0.42,0.0,1.0,0.00,2.380952,44354.474286,185096.803777,0.380952,0.485621,0.857143,0.0,6320.898158,26308.031629,1063


##### ANCHOR

In [ ]:
# Store scores
identity_anchor_scores = []
separability_anchor_scores = []
speed_anchor_seconds = []
# precision_scores = []
accuracy_scores = []
dataset_indeces = []

for i in range(len(X)):
    
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.2, random_state=555)

    rf = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)
    rf.fit(X_train, y_train)

    accuracy = accuracy_score(y_test, rf.predict(X_test))
    print(f"Dataset {folder_names[i]} - Accuracy: {accuracy}")
    dataset_indeces.append(folder_names[i])
    accuracy_scores.append(accuracy)

    explainer = anchor_tabular.AnchorTabularExplainer(
    np.unique(y_train).tolist(),
    X_train.columns.tolist(),
    X_train.values)

    def exp_fn_blk(xtest):
        exp1 = []
        prec = 0
        for i in tqdm.tqdm(range(len(xtest))):
            start_clock = time.time()
            exp = explainer.explain_instance(X_test.values[i], rf.predict, threshold=0.95)
            end_clock = time.time()
            prec += exp.precision()
            calc_time = end_clock - start_clock
            exp_list = [0]*len(X_train.columns)
            for j in exp.features():
                exp_list[j] = 1
            exp1.append(exp_list)
        return np.array(exp1), calc_time, prec

    exp1 = exp_fn_blk(X_test)
    exp2 = exp_fn_blk(X_test)

    precision_scores.append(exp1[2]/len(X_test))
    speed_anchor_seconds.append((exp1[1] + exp2[1]) / 2)
    identity_anchor_scores.append(metrics_rules.calc_identity_rules(exp1[0], exp2[0]))
    separability_anchor_scores.append(metrics_rules.calc_separability_rules(exp1[0]))

In [270]:
df_t = pd.concat([
    pd.Series(dataset_indeces, name='dataset_indeces'),
    #pd.Series(accuracy_scores, name='accuracy_scores'),
    pd.Series(identity_anchor_scores, name='identity_anchor_scores'),
    pd.Series(separability_anchor_scores, name='separability_anchor_scores'),
    pd.Series(speed_anchor_seconds, name='speed_anchor_seconds'),
    #pd.Series(precision_scores, name='precision_scores')
], axis=1)

# df_t.to_csv('all_records_anchor.csv')

# Select only the scores
#df_ts = df_t.head(15)
df_ts['identity_anchor_score'] = df_ts['identity_anchor_scores'].apply(lambda x: x[0])
df_ts['separability_anchor_score'] = df_ts['separability_anchor_scores'].apply(lambda x: x[3])
df_tsd = df_ts.drop(['identity_anchor_scores', 'separability_anchor_scores'], axis=1)

##### Technique testing

In [7]:
# Reading in data
records_anchor = pd.read_csv('records_anchor.csv').drop(['Unnamed: 0'], axis=1)
records_ciu = pd.read_csv('records_ciu.csv').drop(['Unnamed: 0'], axis=1)
records_lime = pd.read_csv('records_lime.csv').drop(['Unnamed: 0'], axis=1)

y_combined = pd.concat([records_lime, records_ciu, records_anchor], axis=1)

folders = y_combined['folder']
y_combined = y_combined.drop(['lime_Dataset', 'folder', 'ciu_Dataset', 'anchor_Dataset'], axis=1)
y_combined['folder'] = folders['folder'].iloc[:, 0]
y_combined.head()

,lime_Fidelity,lime_Identity,lime_Separability,lime_Speed,ciu_Fidelity,ciu_Identity,ciu_Separability,ciu_Speed,anchor_Fidelity,anchor_Identity,anchor_Separability,anchor_Speed,folder
0,0.164644,57.019677,0.2,2.134065,0.05,0.000000,0.0,1.105863,0.164644,57.019677,0.2,2.134065,307
1,0.214557,43.860151,0.0,0.842225,0.10,19.047619,0.0,1.489170,0.214557,43.860151,0.0,0.842225,1067
2,0.180829,98.837384,0.2,1.463465,0.00,100.000000,0.0,0.819695,0.180829,98.837384,0.2,1.463465,50
3,0.163465,10.204481,0.0,2.252382,0.00,6.250000,0.0,1.245967,0.163465,10.204481,0.0,2.252382,32
4,0.127096,20.887676,0.0,1.438171,0.30,0.000000,0.0,2.884910,0.127096,20.887676,0.0,1.438171,1466


In [8]:
# Define metrics
metrics_lime = ['lime_Fidelity', 'lime_Identity', 'lime_Separability', 'lime_Speed']
metrics_CIU = ['ciu_Fidelity', 'ciu_Identity', 'ciu_Separability', 'ciu_Speed']
metrics_anchor = ['anchor_Fidelity', 'anchor_Identity', 'anchor_Separability', 'anchor_Speed']

# Calculate scores for each technique
y_combined['score_lime'] = y_combined[metrics_lime].apply(lambda row: row[0] + row[1] + row[2] - row[3], axis=1)
y_combined['score_CIU'] = y_combined[metrics_CIU].apply(lambda row: row[0] + row[1] + row[2] - row[3], axis=1)
y_combined['score_anchor'] = y_combined[metrics_anchor].apply(lambda row: row[0] + row[1] + row[2] - row[3], axis=1)
y_combined['best_technique'] = y_combined[['score_anchor', 'score_CIU', 'score_anchor']].idxmax(axis=1)

y_selected = y_combined[['folder', 'best_technique']]
y_selected.head()

,folder,best_technique
0,307,score_anchor
1,1067,score_anchor
2,50,score_CIU
3,32,score_anchor
4,1466,score_anchor


In [9]:
merged_df = pd.merge(metadata_df, y_selected, on=['folder'], how='inner')

le = LabelEncoder()
merged_df['best_technique'] = le.fit_transform(merged_df['best_technique'])

merged_df.head()

,attr_to_inst,cat_to_num,freq_class.mean,freq_class.sd,inst_to_attr,max.mean,max.sd,min.mean,min.sd,nr_cor_attr,nr_norm,sd.mean,sd.sd,folder,best_technique
0,0.10,0.0,0.5,0.08,10.000000,728712.000000,894485.604730,1241.000000,2475.504151,0.100000,0.0,233606.804592,286748.820769,1046,0
1,0.74,0.0,0.5,0.44,1.351351,10401.002973,58049.483653,0.223514,0.470010,0.223724,1.0,1533.639358,8540.628907,1049,1
2,0.74,0.0,0.5,0.44,1.351351,9470.734324,51785.848076,0.437568,0.715264,0.340841,0.0,1347.550690,7361.261968,1050,1
3,0.42,0.0,0.5,0.48,2.380952,47852.292857,199422.479258,0.238095,0.425918,0.419048,0.0,6983.118814,28979.323785,1053,1
4,0.42,0.0,1.0,0.00,2.380952,44354.474286,185096.803777,0.380952,0.485621,0.857143,0.0,6320.898158,26308.031629,1063,1


In [10]:
# WIP!

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=555)

Xn = merged_df.iloc[:, :-2]
yn = merged_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(Xn, yn, test_size=0.2, random_state=555)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=555)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.75


Testing

In [34]:
# Metadata generation for test data

columns = ['attr_to_inst',  'cat_to_num',  'freq_class.mean',  'freq_class.sd',  'inst_to_attr',  'max.mean',  'max.sd',  'min.mean',  'min.sd',  'nr_cor_attr',  'nr_norm',  'sd.mean',  'sd.sd']

metadata_df_test = pd.DataFrame(columns=columns)

for i in range(len(X_list_test)):

    mfe = MFE(features=["attr_to_inst", "cat_to_num", "freq_class", "inst_to_attr", "sd", "nr_norm", "nr_cor_attr", "min", "max"])
    mfe.fit(np.array(X_list_test[i]), np.array(y_list_test[i]))
    ft = mfe.extract(
        sd={"ddof": 0},
        nr_norm={"method": "all", "failure": "hard", "threshold": 0.025},
        nr_cor_attr={"threshold": 0.6},
    )

    new = pd.DataFrame(np.array(ft[1]).reshape(1, -1), columns=ft[0])
    metadata_df_test = metadata_df_test.append(new, ignore_index=True)

metadata_df_test['folder'] = X_folder_names_test
metadata_df_test['folder'] = metadata_df_test['folder'].astype(int)
metadata_df_test.head()

,attr_to_inst,cat_to_num,freq_class.mean,freq_class.sd,inst_to_attr,max.mean,max.sd,min.mean,min.sd,nr_cor_attr,nr_norm,sd.mean,sd.sd,folder
0,0.14,0.0,0.1,0.021909,7.142857,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.461235,0.042153,40496
1,0.70,0.0,1.0,0.000000,1.428571,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,42
2,0.10,0.0,1.0,0.000000,10.000000,39.200000,51.394163,17.600000,27.111621,0.000000,1.0,5.666059,7.054412,451
3,0.60,0.0,0.5,0.040000,1.666667,1.000000,0.000000,-0.966667,0.179505,0.020690,0.0,0.731250,0.222545,4534
4,0.64,0.0,1.0,0.000000,1.562500,0.002553,0.003049,-0.001513,0.002409,0.076613,5.0,0.000671,0.000759,4538


In [44]:
rf_classifier.predict(np.array(metadata_df_test.iloc[0, :-1]).reshape(1, -1))

array([1])